In [5]:
import os
import json
import csv
import time
import re
import time
import shutil
# import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

# 儲存需要的標記和將不需要的標記塗黑

In [2]:
fields = ['1Tainan', '2Penghu', '3Taoyuan']

In [4]:
# 南崁.的符號無法decode
bucket = ['220707_060614804_Camera_1', '220707_060617550_Camera_1', '220707_060722774_Camera_1', '220707_060801663_Camera_1', '220707_060950255_Camera_1', '220707_060951671_Camera_1', '220707_060952938_Camera_1', '220707_030343373_Camera_1', '220707_030715653_Camera_1', '220707_030716727_Camera_1', '220707_030717264_Camera_1', '220707_030728389_Camera_1', '220707_030728915_Camera_1', '220707_030728915_Camera_1', '220707_030729441_Camera_1', '220707_030729441_Camera_1', '220707_032755076_Camera_1', '220707_033559873_Camera_1', '220707_061120592_Camera_1', '220707_061123781_Camera_1', '220707_060908688_Camera_1']

# 圓形標記格式的檔名
bucket2 = ['221005_020123663_Camera_0', '221005_020124144_Camera_0', '221005_020124604_Camera_0', '221005_020125040_Camera_0', '221005_020129529_Camera_0', '221005_020130027_Camera_0', '221005_020130531_Camera_0', '221005_020131006_Camera_0', '221005_020131457_Camera_0', '221005_020131893_Camera_0', '221005_020132299_Camera_0', '221005_020132685_Camera_0', '221005_020133047_Camera_0', '221005_020133388_Camera_0', '221005_020133712_Camera_0', '221005_020134020_Camera_0', '220707_030128789_Camera_1', '220707_055132895_Camera_1']

# rect標記格式的檔名
bucket3 = ['220707_030651749_Camera_1', '220707_030652867_Camera_1']

In [5]:
def contains_chinese(text):
    zh_pattern = re.compile(r'[\u4e00-\u9fff]+')
    return bool(zh_pattern.search(text))

In [6]:
t0 = time.time()
for field in fields:
    img_folder_path = f"C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\{field}\\images"
    annotation_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\{field}\\{field}_ori_all.json'
    output_folder_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}'
    output_csv_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\labels_raw.csv'
    os.makedirs(output_folder_path, exist_ok=True)

    with open(annotation_path, 'r', encoding='utf-8') as f:
        annotations = json.load(f)

    valid_filenames = set()

    for image_key, annotation in annotations.items():
        image_filename = annotation["filename"]
        if image_filename.split('.')[0] not in bucket2 and image_filename.split('.')[0] not in bucket3:
            image_path = os.path.join(img_folder_path, image_filename)
            regions = annotation["regions"]
            black_regions = []
            valid_regions = []

            for region in regions:
                all_points_x = region['shape_attributes'].get('all_points_x', [])
                all_points_y = region['shape_attributes'].get('all_points_y', [])
                word = region['region_attributes'].get('word', '')
                category = region['region_attributes']['category']
                

                if (
                    (str(category).split('_')[0] in ['B'] and ('*' not in word and word != '' and contains_chinese(word))) or
                    (str(category).split('_')[0] in ['7', '8', '9', '10', '11', '12'] and ('*' not in word and word != '' and contains_chinese(word))) or
                   (str(category) in ['1', '2', '3', '4', '5', '6'] and (max(all_points_x) - min(all_points_x)) / (max(all_points_y) - min(all_points_y)) >= 1.5 and ('*' not in word and word != '' and contains_chinese(word)))
                ):
                    valid_regions.append(region)
                    valid_filenames.add(image_filename)
                    with open(output_csv_path, 'a', newline='', encoding='big5') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=['raw_filename', 'filename', 'category', 'box', 'words'])
                        if not os.path.isfile(output_csv_path):
                            os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
                            writer.writeheader()
                        writer.writerow({
                            'raw_filename': image_filename,
                            'filename': '',
                            'category': category,
                            'box': list(zip(all_points_x, all_points_y)),
                            'words': word
                        })
                else:
                    black_regions.append(region)

            if image_filename in valid_filenames:
                output_path = os.path.join(output_folder_path, image_filename)
                image = Image.open(image_path)
                ax = plt.gca()
                
                # 找出旋轉方位參數
                try:
                    exif = image._getexif()
                    orientation = exif.get(274, 1)
                except (AttributeError, KeyError, IndexError):
                    orientation = 1

                if orientation == 3:
                    image = image.rotate(180, expand=True)
                elif orientation == 6:
                    image = image.rotate(-90, expand=True)
                elif orientation == 8:
                    image = image.rotate(90, expand=True)
                    
#                 for image_filename in images:
#                     print(f'{image_filename=}')
#                     print('ratio: ', (max(all_points_x) - min(all_points_x)) / (max(all_points_y) - min(all_points_y)))
#                     print(orientation)
                    
                fig, ax = plt.subplots(figsize=(image.width / 100, image.height / 100), dpi=100)

                # 繪製黑色框
                for region in black_regions:
                    all_points_x = region['shape_attributes'].get('all_points_x', [])
                    all_points_y = region['shape_attributes'].get('all_points_y', [])
                    ax.fill(all_points_x, all_points_y, color='black')

#                 # 繪製紅色線
#                 for region in valid_regions:
#                     all_points_x = region['shape_attributes'].get('all_points_x', [])
#                     all_points_y = region['shape_attributes'].get('all_points_y', [])
#                     ax.plot(all_points_x + [all_points_x[0]], all_points_y + [all_points_y[0]], color='red', linewidth=2)
                
                ax.axis('off')
                ax.imshow(np.asarray(image))

                plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
                plt.close()

    print(f"場域: {field}")
    print(f"花費時間: {time.time()-t0}")
    plt.close('all')
    print('-'*80)


場域: 1Tainan
花費時間: 512.7634062767029
--------------------------------------------------------------------------------
場域: 2Penghu
花費時間: 618.0558662414551
--------------------------------------------------------------------------------
場域: 3Taoyuan
花費時間: 729.1006474494934
--------------------------------------------------------------------------------


檢查: 221005_020200383_Camera_0、221005_035614528_Camera_0、221005_034808369_Camera_0、221005_020219414_Camera_0

# 清洗csv檔案(拿掉英文和換行)

In [7]:
import os
import csv

for field in fields:
    input_csv_file = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\labels_raw.csv'
    output_csv_file = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\labels.csv'

    fieldnames = ['raw_filename', 'filename', 'category', 'box', 'words']
    with open(input_csv_file, 'r', encoding='big5') as input_csv, open(output_csv_file, 'w', encoding='big5', newline='') as output_csv:

        csv_writer = csv.writer(output_csv)
        csv_reader = csv.reader(input_csv)
        csv_writer.writerow(fieldnames)

        for row in csv_reader:
            raw_filename, filename, category, box, words = row[0], row[1], row[2], row[3], row[4]
            chinese_words = ''.join([char for char in words if '\u4e00' <= char <= '\u9fff'])
            csv_writer.writerow([raw_filename, filename, category, box, f'{chinese_words}'])

    print(f"{field} 處理完成")
    print('-'*80)

1Tainan 處理完成
--------------------------------------------------------------------------------
2Penghu 處理完成
--------------------------------------------------------------------------------
3Taoyuan 處理完成
--------------------------------------------------------------------------------


#  分配資料夾(8:1:1)

## 處理圖片
先統計，得到資料 result_dict[field]，接著拆分資料集

In [1]:
fields = ['1Tainan', '2Penghu', '3Taoyuan']

In [2]:
import pandas as pd

class CopyImageHelper:
    def __init__(self, data_folder:str):
        self._data_folder = data_folder
        self._mapping_df = pd.DataFrame(columns=["src", "dst", "frequency"])
        
    def copy(self, image_name:str, target_folder:str, frequency:int, *, copy_now=True):
        if self._mapping_df["src"].eq(image_name).any():
            print(f"Skipping duplicate: {image_name}")
            return

        if copy_now:
            src_path = os.path.join(self._data_folder, image_name)
            tar_path = os.path.join(target_folder, image_name)
            shutil.copy(src_path, tar_path)

        self._mapping_df = pd.concat([self._mapping_df, pd.DataFrame({"src": [image_name], "dst": [tar_path], "frequency": [frequency]})], ignore_index=True)
        self._check_the_mapping_is_legal()



    def _check_the_mapping_is_legal(self):
        assert self._mapping_df["src"].is_unique, f"duplicated src {self._mapping_df.loc[self._mapping_df['src'].duplicated(keep=False)]}"

    
    def copy_all(self):
        def copy_file(row):
            try:
                shutil.copy(os.path.join(self._data_folder, row["src"]), row["dst"])
            except FileNotFoundError as e:
                print(f"File not found: {e}")
                pass
        self._mapping_df.apply(copy_file, axis=1)

    def get_mapping_df_data(self):
        return self._mapping_df

In [6]:
import os
import pandas as pd
from collections import Counter
import pandas as pd

for field in fields:
    data_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}'
    output_train_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\train'
    output_val_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\val'
    output_test_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\test'
    labels_file = os.path.join(data_folder, 'labels.csv')

    # 設定訓練集、驗證集和測試集比例 (K:N:M)
    K = 8  # 設定為8表示訓練集佔80%，驗證集和測試集各佔10%
    N = 1
    M = 1

    os.makedirs(output_train_folder, exist_ok=True)
    os.makedirs(output_val_folder, exist_ok=True)
    os.makedirs(output_test_folder, exist_ok=True)

    labels_df = pd.read_csv(labels_file, encoding='big5')

    all_labels = []
    all_raw_filename = []

    for index, row in labels_df.iterrows():
        raw_filename = row['raw_filename']
        label = row['words']
    
        # 將每個標籤的字元拆開，並加入到列表中
        old_all_labels = 0 if not all_labels else len(all_labels)
        label_characters = list(label)
        all_labels.extend(label_characters)

        for _ in range(len(all_labels) - old_all_labels):
            all_raw_filename.append(raw_filename)

    label_frequencies = Counter(all_labels)
    sorted_labels = sorted(label_frequencies, key=label_frequencies.get)

    # 依據字元出現頻率排序檔案名稱
    sorted_files = sorted(all_raw_filename, key=lambda x: label_frequencies[all_labels[all_raw_filename.index(x)]])

    # 計算不同字元的數量
    num_unique_labels = len(sorted_labels)
    print(f'在 {field} 中共有 {num_unique_labels} 個不同字元。')
    
    # 將結果組成一個字典
    result_list = []
    label_result = {
        "field": field,
        "sorted_labels": sorted_labels,
        "label_frequencies": label_frequencies,
        "label_details": []
    }

    # 將每個字元的詳細資訊加入列表
    for label in sorted_labels:
        frequency = label_frequencies[label]
        images_containing_label = [img for img, char in zip(sorted_files, all_labels) if label == char]
        label_result["label_details"].append({
            "label": label,
            "frequency": frequency,
            "images_containing_label": images_containing_label
        })

    result_list.append(label_result)

#     # 印出結果
#     for result in result_list:
#         print(f'{result["field"]=}')
#         for detail in result["label_details"]:
#             print(f"字元: {detail['label']}, 出現頻率: {detail['frequency']}, 原圖: {detail['images_containing_label']}")
#             print('-' * 80)

    # 將結果存入字典
    result_dict = {}
    result_dict[field] = result_list
    
    # we define the helper for copy image
    copy_image_helper = CopyImageHelper(data_folder=data_folder)

    # 每個字元
    cnt = 0
    for detail in result_dict[field][0]['label_details']:
        label = detail['label']
        frequency = detail['frequency']
        images = detail['images_containing_label']

#         print(f'{label=}')
        # 每個字元所存在的原始大圖檔名
        for image in images:

            # 檢查image是否存在於訓練\測試\驗證資料集中 
            if not os.path.exists(os.path.join(output_train_folder, image)) and \
               not os.path.exists(os.path.join(output_test_folder, image)) and \
               not os.path.exists(os.path.join(output_val_folder, image)):
#                 print('-----未重複出現-----')
#                 print(image)

                if frequency == 1:
                    src_path = os.path.join(data_folder, image)
                    tar_path = os.path.join(output_train_folder, image)
                    copy_image_helper.copy(image, output_train_folder, frequency, copy_now=True)
                    cnt += 1

                elif frequency == 2:
                    image1, image2 = images[:]
                    copy_image_helper.copy(image1, output_train_folder, frequency, copy_now=True)
                    copy_image_helper.copy(image2, output_test_folder, frequency, copy_now=True)
                    cnt += 2

                elif 3 <= frequency <= 6:
    #                 random.shuffle(images)
                    val_images = images[:1]
                    test_images = images[1:2]
                    train_images = images[2:]

                    for val_image in val_images:
                        copy_image_helper.copy(val_image, output_val_folder, frequency, copy_now=True)
                        cnt += 1
                    for test_image in test_images:
                        copy_image_helper.copy(test_image, output_test_folder, frequency, copy_now=True)
                        cnt += 1
                    for train_image in train_images:
                        copy_image_helper.copy(train_image, output_train_folder, frequency, copy_now=True)
                        cnt += 1

                else:
                    total_images = len(images)
                    num_test_images = round(total_images * 0.1)
                    num_val_images = round(total_images * 0.1)
                    cnt += frequency


                    test_images = images[:num_test_images]
                    for test_image in test_images:
                        copy_image_helper.copy(test_image, output_test_folder, frequency, copy_now=True)
                        cnt += 1

                    val_images = images[num_test_images:num_test_images + num_val_images]
                    for val_image in val_images:
                        copy_image_helper.copy(val_image, output_val_folder, frequency, copy_now=True)
                        cnt += 1

                    train_images = images[num_test_images + num_val_images:]
                    for train_image in train_images:
                        copy_image_helper.copy(train_image, output_train_folder, frequency, copy_now=True)
                        cnt += 1

            else:
#                 print('-----重複出現-----')
                print(image)
                continue
#         print(f'{cnt=}')
#         print('===========換下一個字==============')

    print(copy_image_helper._mapping_df)
    copy_image_helper.copy_all()

在 1Tainan 中共有 137 個不同字元。
221005_014013697_Camera_0.jpg
221005_014013697_Camera_0.jpg
221005_035604666_Camera_0.jpg
221005_035604666_Camera_0.jpg
221005_035604808_Camera_0.jpg
221005_035604808_Camera_0.jpg
221005_011439700_Camera_0.jpg
Skipping duplicate: 221005_014014442_Camera_0.jpg
Skipping duplicate: 221005_014014442_Camera_0.jpg
Skipping duplicate: 221005_014014442_Camera_0.jpg
221005_014014442_Camera_0.jpg
221005_014014442_Camera_0.jpg
221005_014014442_Camera_0.jpg
Skipping duplicate: 221005_034106502_Camera_0.jpg
221005_014019096_Camera_0.jpg
221005_014019871_Camera_0.jpg
221005_034106502_Camera_0.jpg
221005_014358510_Camera_0.jpg
221005_012359327_Camera_0.jpg
221005_033149445_Camera_0.jpg
Skipping duplicate: 221005_012400071_Camera_0.jpg
221005_012400071_Camera_0.jpg
221005_033927186_Camera_0.jpg
221005_034106650_Camera_0.jpg
221005_012400071_Camera_0.jpg
221005_012400071_Camera_0.jpg
221005_033927186_Camera_0.jpg
221005_034106650_Camera_0.jpg
221005_012400071_Camera_0.jpg
22100

221005_014639672_Camera_0.jpg
221005_014640153_Camera_0.jpg
221005_030126598_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014739250_Camera_0.jpg
221005_014739449_Camera_0.jpg
221005_014740042_Camera_0.jpg
221005_014940459_Camera_0.jpg
221005_014940459_Camera_0.jpg
221005_035336827_Camera_0.jpg
221005_035337083_Camera_0.jpg
221005_035604808_Camera_0.jpg
221005_033753021_Camera_0.jpg
221005_014639185_Camera_0.jpg
221005_014639672_Camera_0.jpg
221005_014640153_Camera_0.jpg
221005_030126598_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014649397_Camera_0.jpg
221005_014739250_Camera_0.jpg
221005_014739449_Camera_0.jpg
221005_014740042_Camera_0.jpg
221005_014940459_Camera_0.jpg
221005_014940459_Camera_0.jpg
221005_035336827_Camera_0.jpg
221005_035337083_Camera_0.jpg
221005_035604808_Camera_0.jpg
Skipping duplicate: 221005_014014703_Camera_0.jpg
221005_014014703_Camera_0.jpg
221005_014014703_Cam

Skipping duplicate: 221005_033319814_Camera_0.jpg
Skipping duplicate: 221005_033320090_Camera_0.jpg
Skipping duplicate: 221005_033330673_Camera_0.jpg
Skipping duplicate: 221005_033330673_Camera_0.jpg
Skipping duplicate: 221005_033330806_Camera_0.jpg
Skipping duplicate: 221005_033330806_Camera_0.jpg
Skipping duplicate: 221005_033331076_Camera_0.jpg
Skipping duplicate: 221005_033331076_Camera_0.jpg
Skipping duplicate: 221005_033331210_Camera_0.jpg
Skipping duplicate: 221005_033331210_Camera_0.jpg
Skipping duplicate: 221005_033331477_Camera_0.jpg
Skipping duplicate: 221005_033403894_Camera_0.jpg
Skipping duplicate: 221005_033403894_Camera_0.jpg
221005_033319814_Camera_0.jpg
221005_033320090_Camera_0.jpg
221005_033330673_Camera_0.jpg
221005_033330673_Camera_0.jpg
221005_033330806_Camera_0.jpg
221005_033330806_Camera_0.jpg
221005_033331076_Camera_0.jpg
221005_033331076_Camera_0.jpg
221005_033331210_Camera_0.jpg
221005_033331210_Camera_0.jpg
221005_033331477_Camera_0.jpg
221005_033403894_Cam

Skipping duplicate: 221005_013854114_Camera_0.jpg
Skipping duplicate: 221005_033926322_Camera_0.jpg
Skipping duplicate: 221005_033926488_Camera_0.jpg
Skipping duplicate: 221005_014648729_Camera_0.jpg
Skipping duplicate: 221005_020352558_Camera_0.jpg
Skipping duplicate: 221005_035334056_Camera_0.jpg
221005_020433244_Camera_0.jpg
221005_020353113_Camera_0.jpg
221005_034128848_Camera_0.jpg
221005_032624804_Camera_0.jpg
221005_034524943_Camera_0.jpg
221005_034525186_Camera_0.jpg
221005_034525308_Camera_0.jpg
221005_014826498_Camera_0.jpg
221005_014014703_Camera_0.jpg
221005_032837316_Camera_0.jpg
221005_034512925_Camera_0.jpg
221005_034808787_Camera_0.jpg
221005_034809757_Camera_0.jpg
221005_035015398_Camera_0.jpg
221005_034525430_Camera_0.jpg
221005_013854417_Camera_0.jpg
221005_014356380_Camera_0.jpg
221005_014357504_Camera_0.jpg
221005_014357504_Camera_0.jpg
221005_014748025_Camera_0.jpg
221005_014748550_Camera_0.jpg
221005_030135854_Camera_0.jpg
221005_014559627_Camera_0.jpg
221005_014

Skipping duplicate: 221005_025316446_Camera_0.jpg
Skipping duplicate: 221005_025042633_Camera_0.jpg
Skipping duplicate: 221005_025042633_Camera_0.jpg
Skipping duplicate: 221005_025042633_Camera_0.jpg
Skipping duplicate: 221005_025043054_Camera_0.jpg
Skipping duplicate: 221005_035335727_Camera_0.jpg
Skipping duplicate: 221005_035335727_Camera_0.jpg
Skipping duplicate: 221005_035336012_Camera_0.jpg
Skipping duplicate: 221005_035337083_Camera_0.jpg
221005_012839271_Camera_0.jpg
221005_012840226_Camera_0.jpg
221005_012840226_Camera_0.jpg
221005_012840226_Camera_0.jpg
221005_012840226_Camera_0.jpg
221005_020252829_Camera_0.jpg
221005_020343919_Camera_0.jpg
221005_024146342_Camera_0.jpg
221005_014739250_Camera_0.jpg
221005_014739449_Camera_0.jpg
221005_014740042_Camera_0.jpg
221005_014740425_Camera_0.jpg
221005_014940459_Camera_0.jpg
221005_014958877_Camera_0.jpg
221005_025057034_Camera_0.jpg
221005_025315432_Camera_0.jpg
221005_025315432_Camera_0.jpg
221005_025315598_Camera_0.jpg
221005_025

...

Skipping duplicate: 220720_232735343_Camera_0.jpg
220721_002206349_Camera_0.jpg
220721_002929951_Camera_0.jpg
220721_002931471_Camera_0.jpg
220721_002932929_Camera_0.jpg
220721_002933875_Camera_0.jpg
220721_002934556_Camera_0.jpg
220721_002935134_Camera_0.jpg
220721_003056461_Camera_0.jpg
220721_003056811_Camera_0.jpg
220720_232735343_Camera_0.jpg
Skipping duplicate: 220720_231745437_Camera_0.jpg
Skipping duplicate: 220720_232733062_Camera_0.jpg
Skipping duplicate: 220720_232733062_Camera_0.jpg
Skipping duplicate: 220720_232734090_Camera_0.jpg
Skipping duplicate: 220720_232734090_Camera_0.jpg
220720_231745437_Camera_0.jpg
220720_231523537_Camera_0.jpg
220720_231524171_Camera_0.jpg
220720_231524799_Camera_0.jpg
220720_232733062_Camera_0.jpg
220720_232733062_Camera_0.jpg
220720_232733062_Camera_0.jpg
220720_232734090_Camera_0.jpg
220720_232734090_Camera_0.jpg
220720_232734090_Camera_0.jpg
220721_002012937_Camera_0.jpg
220720_230725964_Camera_0.jpg
220720_231744669_Camera_0.jpg
220720_231

Skipping duplicate: 220720_232732306_Camera_0.jpg
Skipping duplicate: 220720_232732306_Camera_0.jpg
Skipping duplicate: 220720_232732691_Camera_0.jpg
Skipping duplicate: 220720_232732691_Camera_0.jpg
Skipping duplicate: 220720_232734733_Camera_0.jpg
Skipping duplicate: 220720_232734733_Camera_0.jpg
Skipping duplicate: 220720_232735038_Camera_0.jpg
Skipping duplicate: 220720_232735038_Camera_0.jpg
Skipping duplicate: 220720_231852945_Camera_0.jpg
Skipping duplicate: 220720_232750457_Camera_0.jpg
Skipping duplicate: 220720_232838632_Camera_0.jpg
Skipping duplicate: 220720_232841545_Camera_0.jpg
Skipping duplicate: 220720_234353203_Camera_0.jpg
Skipping duplicate: 220720_234353594_Camera_0.jpg
Skipping duplicate: 220720_225953347_Camera_0.jpg
Skipping duplicate: 220721_002421746_Camera_0.jpg
Skipping duplicate: 220721_002457903_Camera_0.jpg
Skipping duplicate: 220721_002459118_Camera_0.jpg
220720_232910000_Camera_0.jpg
220720_232910000_Camera_0.jpg
220720_232910000_Camera_0.jpg
220721_002

Skipping duplicate: 220707_023452647_Camera_4.jpg
Skipping duplicate: 220707_023751760_Camera_1.jpg
Skipping duplicate: 220707_025147747_Camera_1.jpg
220707_055815982_Camera_1.jpg
220707_023452647_Camera_4.jpg
220707_023751760_Camera_1.jpg
220707_030138538_Camera_1.jpg
220707_024712705_Camera_1.jpg
220707_025147747_Camera_1.jpg
220707_055025684_Camera_1.jpg
220707_033905531_Camera_1.jpg
220707_023452647_Camera_4.jpg
220707_023452647_Camera_4.jpg
220707_023751760_Camera_1.jpg
220707_030138538_Camera_1.jpg
220707_024712705_Camera_1.jpg
220707_025147747_Camera_1.jpg
Skipping duplicate: 220707_060950255_Camera_1.jpg
Skipping duplicate: 220707_060950255_Camera_1.jpg
Skipping duplicate: 220707_025642806_Camera_1.jpg
Skipping duplicate: 220707_025642806_Camera_1.jpg
Skipping duplicate: 220707_025642806_Camera_1.jpg
220707_032505649_Camera_1.jpg
220707_054144804_Camera_1.jpg
220707_030131582_Camera_1.jpg
220707_060950255_Camera_1.jpg
220707_060950255_Camera_1.jpg
220707_025642806_Camera_1.jpg


Skipping duplicate: 220707_060950255_Camera_1.jpg
Skipping duplicate: 220707_060951671_Camera_1.jpg
Skipping duplicate: 220707_032755076_Camera_1.jpg
Skipping duplicate: 220707_030715653_Camera_1.jpg
Skipping duplicate: 220707_031705521_Camera_1.jpg
Skipping duplicate: 220707_031706561_Camera_1.jpg
Skipping duplicate: 220707_024233655_Camera_1.jpg
Skipping duplicate: 220707_024711175_Camera_1.jpg
Skipping duplicate: 220707_025149265_Camera_1.jpg
Skipping duplicate: 220707_025150755_Camera_1.jpg
220707_023943733_Camera_1.jpg
220707_054402598_Camera_1.jpg
220707_055025684_Camera_1.jpg
220707_060614804_Camera_1.jpg
220707_060616271_Camera_1.jpg
220707_060617550_Camera_1.jpg
220707_060722774_Camera_1.jpg
220707_060950255_Camera_1.jpg
220707_060951671_Camera_1.jpg
220707_060411552_Camera_1.jpg
220707_032755076_Camera_1.jpg
220707_030715653_Camera_1.jpg
220707_030715653_Camera_1.jpg
220707_030716190_Camera_1.jpg
220707_025144864_Camera_1.jpg
220707_033230487_Camera_1.jpg
220707_031705521_Cam

Skipping duplicate: 220707_054811709_Camera_1.jpg
Skipping duplicate: 220707_055025684_Camera_1.jpg
Skipping duplicate: 220707_055025684_Camera_1.jpg
Skipping duplicate: 220707_055025684_Camera_1.jpg
Skipping duplicate: 220707_055407171_Camera_1.jpg
Skipping duplicate: 220707_053755977_Camera_1.jpg
Skipping duplicate: 220707_055457776_Camera_1.jpg
Skipping duplicate: 220707_055504045_Camera_1.jpg
Skipping duplicate: 220707_025646150_Camera_1.jpg
Skipping duplicate: 220707_024433292_Camera_1.jpg
Skipping duplicate: 220707_060406292_Camera_1.jpg
Skipping duplicate: 220707_060408837_Camera_1.jpg
Skipping duplicate: 220707_023530886_Camera_4.jpg
Skipping duplicate: 220707_023750177_Camera_1.jpg
Skipping duplicate: 220707_031706561_Camera_1.jpg
Skipping duplicate: 220707_031706561_Camera_1.jpg
Skipping duplicate: 220707_032306078_Camera_1.jpg
Skipping duplicate: 220707_024233655_Camera_1.jpg
Skipping duplicate: 220707_024712705_Camera_1.jpg
Skipping duplicate: 220707_024712705_Camera_1.jpg


## 處理標記檔案

In [7]:
import os
import pandas as pd
import csv
import shutil

fields = ['1Tainan', '2Penghu', '3Taoyuan']
datasets = ['test', 'train', 'val']

for field in fields:
    for dataset in datasets:
        src_csv_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\labels.csv'
        trg_csv_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\{dataset}\\{field}_labels.csv'
        trg_folder_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\{dataset}'

        print(f'{src_csv_path=}')
        print(f'{trg_csv_path=}')
        print(f'{trg_folder_path=}')
        
        # 開啟目標 CSV 檔案
        with open(trg_csv_path, 'w', newline='', encoding='big5') as csvfile:
            fieldnames = ['raw_filename', 'filename', 'category', 'box', 'words']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # 寫入標頭
            writer.writeheader()

            src_df = pd.read_csv(src_csv_path, delimiter=',', encoding='big5')

            # 選擇符合條件的資料
            trg_filenames = os.listdir(trg_folder_path)
            selected_data = src_df[src_df['raw_filename'].isin(trg_filenames)]
            
            print(selected_data)

            if not selected_data.empty:
                # 將選擇的資料寫入目標 CSV 檔案
                writer.writerows(selected_data.to_dict(orient='records'))
                print(f"標記已成功從 {src_csv_path} 複製到 {trg_csv_path}")
            else:
                print('empty!!')
            print('-'*80)

src_csv_path='C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\1Tainan\\labels.csv'
trg_csv_path='C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\1Tainan\\test\\1Tainan_labels.csv'
trg_folder_path='C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\1Tainan\\test'
                       raw_filename  filename category  \
36    221005_013854877_Camera_0.jpg       NaN        1   
37    221005_013854877_Camera_0.jpg       NaN        1   
38    221005_013854877_Camera_0.jpg       NaN        1   
39    221005_013854877_Camera_0.jpg       NaN        1   
40    221005_013854877_Camera_0.jpg       NaN        1   
...                             ...       ...      ...   
1756  221005_034445183_Camera_0.jpg       NaN        3   
1996  221005_035512339_Camera_0.jpg       NaN        2   
1997  221005_035512466_Camera_0.jpg       NaN        2   
2131  221005_035614528_Camera_0.jpg       NaN        8   
2132  221005_035856205_Camera_0.jpg       NaN   

## 檢查是否重疊

In [8]:
import os 

def find_duplicate_images(folder_paths):
    # 儲存所有檔名的字典
    file_names_dict = {}

    for folder_path in folder_paths:
        # 檢查路徑是否存在
        if not os.path.exists(folder_path):
            print(f"資料夾 '{folder_path}' 不存在。")
            continue

        # 取得資料夾中所有檔案
        files = os.listdir(folder_path)

        for file in files:
            # 組合完整檔案路徑
            file_path = os.path.join(folder_path, file)

            # 如果檔案是檔案而非資料夾
            if os.path.isfile(file_path):
                # 如果檔名已經存在於字典中，表示找到重複檔案
                if file in file_names_dict:
                    print(f"重複的檔案: {file}\n位於資料夾: {folder_path} 和 {file_names_dict[file]}")
                    print('-'*80)
                else:
                    # 將檔名和資料夾加入字典
                    file_names_dict[file] = folder_path
for field in fields:
    # 資料夾路徑列表
    folders = [f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\test', f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\train', f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\val']
    # 執行檢查
    find_duplicate_images(folders) 
print('All is done')

重複的檔案: 1Tainan_labels.csv
位於資料夾: C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\1Tainan\train 和 C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\1Tainan\test
--------------------------------------------------------------------------------
重複的檔案: 1Tainan_labels.csv
位於資料夾: C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\1Tainan\val 和 C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\1Tainan\test
--------------------------------------------------------------------------------
重複的檔案: 2Penghu_labels.csv
位於資料夾: C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\2Penghu\train 和 C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\2Penghu\test
--------------------------------------------------------------------------------
重複的檔案: 2Penghu_labels.csv
位於資料夾: C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\2Penghu\val 和 C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\2Penghu\test
---------------------------------------------------------

## 檢查是否漏掉

In [9]:
for field in fields:
    data_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}'
    train_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\train'
    val_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\val'
    test_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\test'

    # 獲取每個目錄的檔案列表
    data_files = set(os.listdir(data_folder))
    train_files = set(os.listdir(train_folder))
    val_files = set(os.listdir(val_folder))
    test_files = set(os.listdir(test_folder))

    # 找出差異
    missing = data_files - (train_files | val_files | test_files)

    # 打印結果
    print(f"Files missing in {field} folder: {missing}")

Files missing in 1Tainan folder: {'train', 'labels.csv', 'test', 'labels_raw.csv', 'val'}
Files missing in 2Penghu folder: {'train', 'labels.csv', 'test', 'labels_raw.csv', 'val'}
Files missing in 3Taoyuan folder: {'train', 'labels.csv', 'test', 'labels_raw.csv', 'val'}


# 合併資料夾

## 合併圖片

In [10]:
import os
import pandas as pd
from shutil import copyfile

fields = ['1Tainan', '2Penghu', '3Taoyuan']
output_train_folder = 'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\train'
output_val_folder = 'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\val'
output_test_folder = 'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\test'

# 創建輸出資料夾
os.makedirs(output_train_folder, exist_ok=True)
os.makedirs(output_val_folder, exist_ok=True)
os.makedirs(output_test_folder, exist_ok=True)

# 合併各field的資料
for field in fields:
    field_train_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\train'
    field_val_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\val'
    field_test_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{field}\\test'

    # 將field的training dataset複製到統一的training資料夾，避免檔名覆蓋問題
    for filename in os.listdir(field_train_folder):
        source_image = os.path.join(field_train_folder, filename)
        destination_image = os.path.join(output_train_folder, f'{filename}')
        copyfile(source_image, destination_image)

    # 將field的validation dataset複製到統一的validation資料夾，避免檔名覆蓋問題
    for filename in os.listdir(field_val_folder):
        source_image = os.path.join(field_val_folder, filename)
        destination_image = os.path.join(output_val_folder, f'{filename}')
        copyfile(source_image, destination_image)

    # 將field的test dataset複製到統一的testing資料夾，避免檔名覆蓋問題
    for filename in os.listdir(field_test_folder):
        source_image = os.path.join(field_test_folder, filename)
        destination_image = os.path.join(output_test_folder, f'{filename}')
        copyfile(source_image, destination_image)
print('all is done')

all is done


## 合併標記檔(csv)

In [11]:
# 合併標記檔
datasets = ['test', 'train', 'val']
for dataset in datasets:
    labels_dfs = []
    output_combined_csv = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{dataset}\\labels.csv'
    print('output: ', output_combined_csv)
    
    for field in fields:
        field_labels_file = os.path.join(f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{dataset}', f'{field}_labels.csv')
        labels_df = pd.read_csv(field_labels_file, encoding='big5')
        labels_df['filename'] = labels_df['filename'].apply(lambda x: f'{field}_{x}')  # 將filename加上field的前綴
        labels_dfs.append(labels_df)

    # 將標記檔合併
    merged_labels_df = pd.concat(labels_dfs, ignore_index=True)
    
    # 將特定列轉換為字符串型別
    columns_to_string = ['raw_filename', 'filename', 'category', 'box', 'words']
    merged_labels_df[columns_to_string] = merged_labels_df[columns_to_string].astype(str)
    
    # 將合併後的標記檔儲存到統一的training資料夾
    merged_labels_df.to_csv(output_combined_csv, index=False, encoding='big5')


output:  C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\test\labels.csv
output:  C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\train\labels.csv
output:  C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\x\val\labels.csv


## 檢查是否csv中的檔案都有存在於目標資料夾中

In [12]:
datasets = ['test', 'train', 'val']
for dataset in datasets:
    output_combined_csv = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{dataset}\\labels.csv'
    output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{dataset}'

    with open(output_combined_csv, 'r') as csv_file:
        next(csv_file)
        # 假設檔名是第一行
        file_names = [line.split(',')[0].strip() for line in csv_file]

        for file_name in file_names:
            file_path = os.path.join(output_folder, file_name)
            if os.path.exists(file_path):
                pass
#                 print(f'{file_name} exists in {output_folder}')
            else:
                print(f'{file_name} does not exist in {output_folder}')
print('all is done')

all is done
